## BART (Encoder-Decoder Style Model)

BART, or Bidirectional AutoRegressive Transformer found in [this](https://arxiv.org/pdf/1910.13461v1.pdf) paper, is a Encoder-Decoder style model that leverages the traditional architecture found in the "Attention is All You Need" paper. They make a simple modification to the activation function from ReLU to GeLU.

This model excels at a number of tasks, including but not limited to: Machine Translation, Summarization, Categorization of Input Sentences, and Question Answering.

We'll showcase BART with a Text Summarization fine-tuning task today.

In [ ]:
!pip install rouge-score evaluate transformers accelerate -qU

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, set_seed
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

import datasets
from datasets import load_metric, Dataset
from datasets import DatasetDict

First up, we'll load our model and tokenizer!

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "facebook/bart-base"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

We'll be using the `billsum` dataset from Hugging Face which you can be found [here](https://huggingface.co/datasets/billsum/viewer/default).

Each of the rows contains a block of text from a legal bill - and then a plain english summary.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("billsum", split="ca_test")

In [ ]:
dataset

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

We'll create a train/test/eval split to train our model.

In [ ]:
import math

total_rows = 500
test_val_ratio = 0.2

val_rows = total_rows + math.floor(total_rows * test_val_ratio)
test_rows = val_rows + math.floor(total_rows * test_val_ratio)

subset_dataset = datasets.DatasetDict(
    {
        "train" : Dataset.from_dict(dataset[:total_rows]),
        "validation" : Dataset.from_dict(dataset[total_rows:val_rows]),
        "test" : Dataset.from_dict(dataset[val_rows:test_rows])
    }
)

In [ ]:
subset_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 100
    })
})

We need to preprocess our data into tokenized representations.

These tokenized representations are what the model will actually see during training!

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = subset_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

We can remove all unessecary text columns.

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset.column_names)

We'll set up an evaluation pipeline that will help us monitor our model's performance!

In [ ]:
!pip install nltk -qU

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: round(v, 4) for k, v in result.items()}

Now we can finally get to training!

We're going to train with the `Seq2Seq` objective as we're trying to convert one long sequence into a shorter sequence.

In [ ]:
batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_ckpt

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-CNN-DailyNews",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.676700,1.891111,0.174500,0.091500,0.153600,0.164400
2,2.069100,1.590447,0.177700,0.100300,0.157900,0.167700
3,1.804700,1.365242,0.177800,0.102900,0.158700,0.166300
4,1.634500,1.231695,0.195900,0.122600,0.175100,0.184200
5,1.483700,1.109857,0.201500,0.126500,0.179600,0.191100
6,1.390400,1.026705,0.200400,0.127800,0.179900,0.189300
7,1.287600,0.978820,0.197800,0.130700,0.178400,0.187800
8,1.257800,0.958415,0.197700,0.132100,0.179200,0.188400


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=504, training_loss=1.6920497928346907, metrics={'train_runtime': 781.797, 'train_samples_per_second': 5.116, 'train_steps_per_second': 0.645, 'total_flos': 2438945832960000.0, 'train_loss': 1.6920497928346907, 'epoch': 8.0})

Now we can push our model to the Hugging Face Hub to test and play around with!

In [ ]:
!pip install huggingface-hub -qU

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("ai-maker-space/Transformers-Workshop-BART-Summarization")

You can check out the final model [here](https://huggingface.co/ai-maker-space/Transformers-Workshop-BART-Summarization?text=SECTION+1.+ENVIRONMENTAL+INFRASTRUCTURE.+%28a%29+Jackson+County%2C+Mississippi.--Section+219+of+the+Water+Resources+Development+Act+of+1992+%28106+Stat.+4835%3B+110+Stat.+3757%29+is+amended--+%281%29+in+subsection+%28c%29%2C+by+striking+paragraph+%285%29+and+inserting+the+following%3A+%60%60%285%29+Jackson+county%2C+mississippi.--Provision+of+an+alternative+water+supply+and+a+project+for+the+elimination+or+control+of+combined+sewer+overflows+for+Jackson+County%2C+Mississippi.%27%27%3B+and+%282%29+in+subsection+%28e%29%281%29%2C+by+striking+%60%60%2410%2C000%2C000%27%27+and+inserting+%60%60%2420%2C000%2C000%27%27.+%28b%29+Manchester%2C+New+Hampshire.--Section+219%28e%29%283%29+of+the+Water+Resources+Development+Act+of+1992+%28106+Stat.+4835%3B+110+Stat.+3757%29+is+amended+by+striking+%60%60%2410%2C000%2C000%27%27+and+inserting+%60%60%2420%2C000%2C000%27%27.+%28c%29+Atlanta%2C+Georgia.--Section+219%28f%29%281%29+of+the+Water+Resources+Development+Act+of+1992+%28106+Stat.+4835%3B+113+Stat.+335%29+is+amended+by+striking+%60%60%2425%2C000%2C000+for%27%27.+%28d%29+Paterson%2C+Passaic+County%2C+and+Passaic+Valley%2C+New+Jersey.--+Section+219%28f%29%282%29+of+the+Water+Resources+Development+Act+of+1992+%28106+Stat.+4835%3B+113+Stat.+335%29+is+amended+by+striking+%60%60%2420%2C000%2C000+for%27%27.+%28e%29+Elizabeth+and+North+Hudson%2C+New+Jersey.--Section+219%28f%29+of+the+Water+Resources+Development+Act+of+1992+%28106+Stat.+4835%3B+113+Stat.+335%29+is+amended--+%281%29+in+paragraph+%2833%29%2C+by+striking+%60%60%2420%2C000%2C000%27%27+and+inserting+%60%60%2410%2C000%2C000%27%27%3B+and+%282%29+in+paragraph+%2834%29--+%28A%29+by+striking+%60%60%2410%2C000%2C000%27%27+and+inserting+%60%60%2420%2C000%2C000%27%27%3B+and+%28B%29+by+striking+%60%60in+the+city+of+North+Hudson%27%27+and+inserting+%60%60for+the+North+Hudson+Sewerage+Authority%27%27.+SEC.+2.+UPPER+MISSISSIPPI+RIVER+ENVIRONMENTAL+MANAGEMENT+PROGRAM.+Section+1103%28e%29%285%29+of+the+Water+Resources+Development+Act+of+1986+%2833+U.S.C.+652%28e%29%285%29%29+%28as+amended+by+section+509%28c%29%283%29+of+the+Water+Resources+Development+Act+of+1999+%28113+Stat.+340%29%29+is+amended+by+striking+%60%60paragraph+%281%29%28A%29%28i%29%27%27+and+inserting+%60%60paragraph+%281%29%28B%29%27%27.+SEC.+3.+DELAWARE+RIVER%2C+PENNSYLVANIA+AND+DELAWARE.+Section+346+of+the+Water+Resources+Development+Act+of+1999+%28113+Stat.+309%29+is+amended+by+striking+%60%60economically+acceptable%27%27+and+inserting+%60%60environmentally+acceptable%27%27.+SEC.+4.+PROJECT+REAUTHORIZATIONS.+Section+364+of+the+Water+Resources+Development+Act+of+1999+%28113+Stat.+313%29+is+amended--+%281%29+by+striking+%60%60Each%27%27+and+all+that+follows+through+the+colon+and+inserting+the+following%3A+%60%60Each+of+the+following+projects+is+authorized+to+be+carried+out+by+the+Secretary%2C+and+no+construction+on+any+such+project+may+be+initiated+until+the+Secretary+determines+that+the+project+is+technically+sound%2C+environmentally+acceptable%2C+and+economically+justified%3A%27%27%3B+%282%29+by+striking+paragraph+%281%29%3B+and+%283%29+by+redesignating+paragraphs+%282%29+through+%286%29+as+paragraphs+%281%29+through+%285%29%2C+respectively.+SEC.+5.+SHORE+PROTECTION.+Section+103%28d%29%282%29%28A%29+of+the+Water+Resources+Development+Act+of+1986+%2833+U.S.C.+2213%28d%29%282%29%28A%29%29+%28as+amended+by+section+215%28a%29%282%29+of+the+Water+Resources+Development+Act+of+1999+%28113+Stat.+292%29%29+is+amended+by+striking+%60%60or+for+which+a+feasibility+study+is+completed+after+that+date%2C%27%27+and+inserting+%60%60except+for+a+project+for+which+a+District+Engineer%27s+Report+is+completed+by+that+date%2C%27%27.+SEC.+6.+COMITE+RIVER%2C+LOUISIANA.+Section+371+of+the+Water+Resources+Development+Act+of+1999+%28113+Stat.+321%29+is+amended--+%281%29+by+inserting+%60%60%28a%29+In+General.--%27%27+before+%60%60The%27%27%3B+and+%282%29+by+adding+at+the+end+the+following%3A+%60%60%28b%29+Crediting+of+Reduction+in+Non-Federal+Share.--The+project+cooperation+agreement+for+the+Comite+River+Diversion+Project+shall+include+a+provision+that+specifies+that+any+reduction+in+the+non-+Federal+share+that+results+from+the+modification+under+subsection+%28a%29+shall+be+credited+toward+the+share+of+project+costs+to+be+paid+by+the+Amite+River+Basin+Drainage+and+Water+Conservation+District.%27%27.+SEC.+7.+CHESAPEAKE+CITY%2C+MARYLAND.+Section+535%28b%29+of+the+Water+Resources+Development+Act+of+1999+%28113+Stat.+349%29+is+amended+by+striking+%60%60the+city+of+Chesapeake%27%27+each+place+it+appears+and+inserting+%60%60Chesapeake+City%27%27.+SEC.+8.+CONTINUATION+OF+SUBMISSION+OF+CERTAIN+REPORTS+BY+THE+SECRETARY+OF+THE+ARMY.+%28a%29+Recommendations+of+Inland+Waterways+Users+Board.--Section+302%28b%29+of+the+Water+Resources+Development+Act+of+1986+%2833+U.S.C.+2251%28b%29%29+is+amended+in+the+last+sentence+by+striking+%60%60The%27%27+and+inserting+%60%60Notwithstanding+section+3003+of+Public+Law+104-66+%2831+U.S.C.+1113+note%3B+109+Stat.+734%29%2C+the%27%27.+%28b%29+List+of+Authorized+but+Unfunded+Studies.--Section+710%28a%29+of+the+Water+Resources+Development+Act+of+1986+%2833+U.S.C.+2264%28a%29%29+is+amended+in+the+first+sentence+by+striking+%60%60Not%27%27+and+inserting+%60%60Notwithstanding+section+3003+of+Public+Law+104-66+%2831+U.S.C.+1113+note%3B+109+Stat.+734%29%2C+not%27%27.+%28c%29+Reports+on+Participation+of+Minority+Groups+and+Minority-Owned+Firms+in+Mississippi+River-Gulf+Outlet+Feature.--Section+844%28b%29+of+the+Water+Resources+Development+Act+of+1986+%28100+Stat.+4177%29+is+amended+in+the+second+sentence+by+striking+%60%60The%27%27+and+inserting+%60%60Notwithstanding+section+3003+of+Public+Law+104-66+%2831+U.S.C.+1113+note%3B+109+Stat.+734%29%2C+the%27%27.+%28d%29+List+of+Authorized+but+Unfunded+Projects.--Section+1001%28b%29%282%29+of+the+Water+Resources+Development+Act+of+1986+%2833+U.S.C.+579a%28b%29%282%29%29+is+amended+in+the+first+sentence+by+striking+%60%60Every%27%27+and+inserting+%60%60Notwithstanding+section+3003+of+Public+Law+104-66+%2831+U.S.C.+1113+note%3B+109+Stat.+734%29%2C+every%27%27.+SEC.+9.+AUTHORIZATIONS+FOR+PROGRAM+PREVIOUSLY+AND+CURRENTLY+FUNDED.+%28a%29+Program+Authorization.--The+program+described+in+subsection+%28c%29+is+hereby+authorized.+%28b%29+Authorization+of+Appropriations.--Funds+are+hereby+authorized+to+be+appropriated+for+the+Department+of+Transportation+for+the+program+authorized+in+subsection+%28a%29+in+amounts+as+follows%3A+%281%29+Fiscal+year+2000.--For+fiscal+year+2000%2C+%2410%2C000%2C000.+%282%29+Fiscal+year+2001.--For+fiscal+year+2001%2C+%2410%2C000%2C000.+%283%29+Fiscal+year+2002.--For+fiscal+year+2002%2C+%247%2C000%2C000.+%28c%29+Applicability.--The+program+referred+to+in+subsection+%28a%29+is+the+program+for+which+funds+appropriated+in+title+I+of+Public+Law+106-+69+under+the+heading+%60%60FEDERAL+RAILROAD+ADMINISTRATION%27%27+are+available+for+obligation+upon+the+enactment+of+legislation+authorizing+the+program.+Speaker+of+the+House+of+Representatives.+Vice+President+of+the+United+States+and+President+of+the+Senate.)!

## BERT (Encoder Only Architecture)

We'll be using BERT (found in [this paper]()) as our example of an Encoder-only transformer model.

BERT-style models excel at Sentiment Analysis, Question Answering, Text Prediction, and other language comprehension tasks.

The [20 Newsgroups dataset](http://qwone.com/~jason/20Newsgroups/) is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.

[Here are some details](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset) about the dataset from Scikit Learn!

Let's load the data and get it into a usable format!

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset = "train")
test = fetch_20newsgroups(subset = "test")

Have a look around the data! Take note of things like data types, column names, and everything else!

Also take note of how many classes there are in our labels and mark it down in the cell below!

In [ ]:
NUM_LABELS = 20

In [ ]:
import pandas as pd

X, y = pd.Series(train["data"]), pd.Series(train["target"])
X_test, y_test = pd.Series(test["data"]), pd.Series(test["target"])

Now that we have our raw data - let's convert that into some pd.Series objects using pandas!

Now let's get the Hugging Face datasets ([documentation here](https://huggingface.co/docs/datasets/index)) library so we can convert our data into a more usable format.

In [ ]:
train_df = pd.DataFrame({
    "text" : X,
    "label" : y
})

test_df = pd.DataFrame({
    "text" : X_test,
    "label" : y_test
})

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

Now we can cast our label columns to datasets.features.ClassLabel objects using class_encode_column! (documentation [here](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.class_encode_column))

In [ ]:
train_ds = train_ds.class_encode_column("label")
test_ds = test_ds.class_encode_column("label")

Stringifying the column:   0%|          | 0/11314 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/11314 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/7532 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/7532 [00:00<?, ? examples/s]

We'll want to first convert our separate series objects into a combined pd.DataFrame with columns: text and label, for our Xs and ys respectively.

After that, it's as easy as loading the pd.DataFrame into a Dataset object!

In [ ]:
data_dsd = train_ds.train_test_split(test_size=0.1, seed=19, stratify_by_column="label")

In [ ]:
data_dsd['validation'] = data_dsd['test']
data_dsd['test'] = test_ds

In [ ]:
data_dsd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1132
    })
})

In this task, we'll be fine-tuning a simple classifier for our above data using Hugging Face's transformers library (documentation [here](https://huggingface.co/docs/accelerate/index)) as well as the accelerate library. (documentation [here](https://huggingface.co/docs/transformers/index))

Before we dive in, let's take a pit stop to discuss what fine-tuning is - in broad strokes.

- Fine-tuning is a transfer learning approach where a pre-trained machine learning model is further trained on new data, often to specialize in a certain task. This process can involve training the entire network or only a subset of it, with untrained layers remaining 'frozen'.

- This method is prevalent in Natural Language Processing (NLP) and convolutional neural networks. In the latter, early layers capturing lower-level features are typically frozen, while in NLP, large models like GPT-2 are fine-tuned for specific tasks, improving their performance. However, full fine-tuning can be computationally costly and might lead to overfitting.

- Although fine-tuning is commonly executed through supervised learning, it can also be done using weak supervision or reinforcement learning. For instance, language models like ChatGPT and Sparrow are fine-tuned using reinforcement learning from human feedback.

Okay, so now that we have had a brief overview of what fine-tuning actually is - let's set ourselves up to do some!

In [ ]:
bert_model_id = "distilbert-base-uncased"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(bert_model_id)

In [ ]:
MAX_LEN = 256

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=MAX_LEN)

In [ ]:
tokenized_text = data_dsd.map(preprocess_function, batched=True)

Map:   0%|          | 0/10182 [00:00<?, ? examples/s]

Map:   0%|          | 0/7532 [00:00<?, ? examples/s]

Map:   0%|          | 0/1132 [00:00<?, ? examples/s]

In [ ]:
tokenized_text

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1132
    })
})

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")

We'll want to include the attention_mask, input_ids, and label for each set - as well as shuffling the training set.

In [ ]:
BATCH_SIZE = 16

tf_train_set = tokenized_text["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_text["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids","label"],
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )

tf_test_set = tokenized_text["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids","label"],
    shuffle=False,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import create_optimizer

EPOCHS = 3
batches_per_epoch = len(tokenized_text["train"]) // BATCH_SIZE
total_train_steps = int(batches_per_epoch * EPOCHS)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

my_bert = TFAutoModelForSequenceClassification.from_pretrained(bert_model_id, num_labels=NUM_LABELS)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

We're going to use the naive example of accuracy for this notebook - but feel free to use whatever metric you believe will work best.

In [ ]:
my_bert.compile(optimizer=optimizer,  metrics=['accuracy'])

In [ ]:
%%time
my_bert.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
637/637 [==============================] - 124s 160ms/step - loss: 1.3256 - accuracy: 0.6790 - val_loss: 0.6010 - val_accuracy: 0.8295
Epoch 2/3
637/637 [==============================] - 98s 153ms/step - loss: 0.4440 - accuracy: 0.8803 - val_loss: 0.4090 - val_accuracy: 0.8816
Epoch 3/3
637/637 [==============================] - 97s 153ms/step - loss: 0.2683 - accuracy: 0.9357 - val_loss: 0.3717 - val_accuracy: 0.8984
CPU times: user 4min 32s, sys: 11.8 s, total: 4min 44s
Wall time: 5min 18s


In [ ]:
bert_loss, bert_acc = my_bert.evaluate(tf_test_set)

471/471 [==============================] - 25s 52ms/step - loss: 0.5596 - accuracy: 0.8388


In [ ]:
HUGGINGFACE_ACCT_NAME = "ai-maker-space"
MODEL_NAME = "Transformers-Workshop-BERT-NewsGroupClassification"

In [ ]:
my_bert.push_to_hub(f"{HUGGINGFACE_ACCT_NAME}/{MODEL_NAME}")
tokenizer.push_to_hub(f"{HUGGINGFACE_ACCT_NAME}/{MODEL_NAME}")

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ai-maker-space/Transformers-Workshop-BERT-NewsGroupClassification/commit/e40b26003edba3ef2737f263c015eb7727a6d46b', commit_message='Upload tokenizer', commit_description='', oid='e40b26003edba3ef2737f263c015eb7727a6d46b', pr_url=None, pr_revision=None, pr_num=None)

## GPT-2 (Decoder-only Architecture)

Next up, and perhaps more importantly, we have our GPT-style models. These models are built from decoder-only architecture and work in an autoregressive fashion. Essentially, these models generate tokens one-by-one in sequence based on the tokens that precede it.

You can read more about GPT-2 in [this paper](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)

Decoder-only Architectures excel at text generation, language modeling, and creative writing.

We're going to spending a lot of time on this style architecture in our course - so we'll be zooming through this section!

We'll be leveraging a lyric dataset to fine-tune our GPT-2-small model, you can find the dataset [here]()

In [ ]:
lyric_dataset = load_dataset("brunokreiner/genius-lyrics")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
lyric_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'lyrics', 'is_english', 'genres_list', 'popularity', 'release_date', 'artist_id', 'artist_name', 'artist_popularity', 'artist_followers', 'artist_picture_url'],
        num_rows: 480855
    })
})

In [ ]:
import math

total_rows = 500
test_val_ratio = 0.2

val_rows = total_rows + math.floor(total_rows * test_val_ratio)
test_rows = val_rows + math.floor(total_rows * test_val_ratio)

subset_dataset = datasets.DatasetDict(
    {
        "train" : Dataset.from_dict(lyric_dataset["train"][:total_rows]),
        "validation" : Dataset.from_dict(lyric_dataset["train"][total_rows:val_rows]),
        "test" : Dataset.from_dict(lyric_dataset["train"][val_rows:test_rows])
    }
)

In [ ]:
subset_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'lyrics', 'is_english', 'genres_list', 'popularity', 'release_date', 'artist_id', 'artist_name', 'artist_popularity', 'artist_followers', 'artist_picture_url'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'id', 'lyrics', 'is_english', 'genres_list', 'popularity', 'release_date', 'artist_id', 'artist_name', 'artist_popularity', 'artist_followers', 'artist_picture_url'],
        num_rows: 100
    })
    test: Dataset({
        features: ['Unnamed: 0', 'id', 'lyrics', 'is_english', 'genres_list', 'popularity', 'release_date', 'artist_id', 'artist_name', 'artist_popularity', 'artist_followers', 'artist_picture_url'],
        num_rows: 100
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

gpt_model_id = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(gpt_model_id)
model = AutoModelForCausalLM.from_pretrained(gpt_model_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["lyrics"])

In [ ]:
tokenized_datasets = subset_dataset.map(tokenize_function, batched=True, num_proc=1)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(lyric_dataset["train"].column_names)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
block_size = int(tokenizer.model_max_length / 4)

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
model_name = "lyric-gpt"

num_train_epochs = 30

training_args = TrainingArguments(
    f"output/{model_name}",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    learning_rate=1.00e-4,
    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    save_total_limit=10,
    save_strategy='epoch',
    save_steps=1,
    report_to=None,
    logging_steps=5,
    do_eval=True,
    eval_steps=1,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"]
)

In [ ]:
from transformers import get_cosine_schedule_with_warmup
train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader)
trainer.create_optimizer_and_scheduler(num_train_steps)
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
      trainer.optimizer,
      num_warmup_steps=0,
      num_training_steps=num_train_steps
)

trainer.model.config.task_specific_params['text-generation'] = {
                    'do_sample': True,
                    'min_length': 100,
                    'max_length': 200,
                    'temperature': 1.,
                    'top_p': 0.95,
                    }

In [ ]:
import torch
torch.cuda.empty_cache()

data = trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.532700,3.188484
2,2.898800,3.198559
3,3.019200,3.247682
4,2.523200,3.331820
5,2.309800,3.389926
6,2.141200,3.532782
7,2.352600,3.632200
8,2.060000,3.783356
9,1.947700,3.888714
10,1.615600,4.052728


Now that we've trained our model - let's upload it to the Hugging Face Hub!

In [ ]:
trainer.push_to_hub("ai-maker-space/Transformers-Workshop-GPT-Generation")

In [ ]:
tokenizer.push_to_hub("ai-maker-space/Transformers-Workshop-GPT-Generation")

CommitInfo(commit_url='https://huggingface.co/ai-maker-space/Transformers-Workshop-GPT-Generation/commit/9e394b5dc49eb1a6d07d838f4013635954fb143e', commit_message='Upload tokenizer', commit_description='', oid='9e394b5dc49eb1a6d07d838f4013635954fb143e', pr_url=None, pr_revision=None, pr_num=None)

You can find the model [here](https://huggingface.co/ai-maker-space/Transformers-Workshop-GPT-Generation?text=I+am)!

Let's see how the generation works!

In [ ]:
start = "I am"
num_sequences =  5
min_length =  100
max_length =   160
temperature = 1
top_p = 0.95
top_k = 50
repetition_penalty =  1.4

encoded_prompt = tokenizer(start, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)
output_sequences = trainer.model.generate(
                        input_ids=encoded_prompt,
                        max_length=max_length,
                        min_length=min_length,
                        temperature=float(temperature),
                        top_p=float(top_p),
                        top_k=int(top_k),
                        do_sample=True,
                        repetition_penalty=repetition_penalty,
                        num_return_sequences=num_sequences
                        )

generated_sequences = []

for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
        generated_sequences.append(text.strip())

for generation in generated_sequences:
  print(generation)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am a little rough with my body on the rocks as you grow old it s going to be an easy day all of your friends are trying not come down just for me i m ready and don t need no stress in this world there is something special about seeing people around yourself that makes sense why can we fight against each other but now one step at every corner where they hold hands together when their eyes meet mine if ever someone saw what I did yesterday she would have given up her faith cause somebody told them oh ma la do niggas get hurt so runnin bout em running through life yeah like these nigga dudes wanna hear ya say things back home ain d seein shit too bad some will tell yall lies right well sometimes its harder than others though let's figure out how many miles
I am the way you see me i just know how to get what I want and it s not a battle or war if your love is wrong at all make sure that everything comes together oh but he can never do so in my dreams they don t always work for him  yeah 

# Attention is All You Need, Right?

We'll begin by looking at how the basic Transformer Block is set up using PyTorch - and to do that, we'll start with our dependencies!

We'll start with the classic image of the Transformer from the classic paper ["Attention is All You Need"](https://arxiv.org/pdf/1706.03762.pdf).

![img](https://i.imgur.com/4pA8cS6.png)

## Multi-Head Attention

The first step to creating the transformer is straightforward enough: We need to code up that Attention mechanism!

We need two components to make this happen:

1. Scaled Dot-Product Attention
2. Multi-Head Attention

Let's look at their respective images from the paper!

![img](https://i.imgur.com/1Sp9EXp.png)

The basic idea is as follows:

We allow different Attention Heads to attend to different parts of the sequence with different representation subspaces.

All those words to say that each of our Attention Heads will care about different things throughout the course of training - as the old adage goes: Many heads are better than one!

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import copy

Now we can create our MultiHeadAttention Module!

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Input Dimension of Model
        self.d_model = d_model

        # Number of Heads (h)
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        # Q Linear Layer
        self.W_q = nn.Linear(d_model, d_model)

        # K Linear Layer
        self.W_k = nn.Linear(d_model, d_model)

        # V Linear Layer
        self.W_v = nn.Linear(d_model, d_model)

        # Output Linear Layer
        self.W_o = nn.Linear(d_model, d_model)

    ### Left Side of the Above Image
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    ### Right Side of the Above Image
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output